In [ ]:
import requests
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

spark = SparkSession.builder.appName("GetTickersTypeDataFromApi").getOrCreate()
url = "https://api.polygon.io/v3/reference/tickers/types?asset_class=stocks&apiKey=VOYRq_vnd9soI0AhCO8FQpK4auZ6ppc3"

ticker_types_schema = StructType([
    StructField("asset_class", StringType(), True),
    StructField("code", StringType(), True),
    StructField("description", StringType(), True),
    StructField("locale", StringType(), True)
])

response = requests.get(url)

ticker_types_df = spark.createDataFrame(response.json()["results"], schema = ticker_types_schema)


ticker_types_df.write.format("parquet").\
    option("compression", "snappy").\
    mode("append").\
        save("gs://stocks-pipeline/raw-data/ticker_types")

spark.stop()
